# Conversational agent

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.tools import tool

In [3]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [4]:
tools = [search_wikipedia]

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [19]:
functions = [format_tool_to_openai_function(f) for f in tools]

model = ChatOpenAI(temperature=0).bind(functions=functions)

In [9]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [20]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [21]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [22]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [23]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [24]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?')],
 'output': 'Nice to meet you, Bob! How can I assist you today?'}

In [25]:
agent_executor.invoke({"input": "whats my name"})



> Entering new AgentExecutor chain...
Your name is Bob.

> Finished chain.


{'input': 'whats my name',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob.')],
 'output': 'Your name is Bob.'}

In [27]:
agent_executor.invoke({"input": "what is artemis 1 mission?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Artemis 1 mission'}`


Page: Artemis 1
Summary: Artemis 1, officially Artemis I and formerly Exploration Mission-1 (EM-1), was an uncrewed Moon-orbiting mission. As the first major spaceflight of NASA's Artemis program, Artemis 1 marked the agency's return to lunar exploration after the conclusion of the Apollo program five decades earlier. It was the first integrated flight test of the Orion spacecraft and Space Launch System (SLS) rocket, and its main objective was to test the Orion spacecraft, especially its heat shield, in preparation for subsequent Artemis missions. These missions seek to reestablish a human presence on the Moon and demonstrate technologies and business approaches needed for future scientific studies, including exploration of Mars.
The Orion spacecraft for Artemis 1 was stacked on October 20, 2021, and on August 17, 2022, the fully stacked vehicle was rolled out for launch after

{'input': 'what is artemis 1 mission?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob.'),
  HumanMessage(content='whats the weather in sf?'),
  AIMessage(content="I'm sorry, I can't provide real-time weather updates. You can check the weather in San Francisco by using a weather website or app. Let me know if you need help with anything else!"),
  HumanMessage(content='what is artemis 1 mission?'),
  AIMessage(content="Artemis 1, officially known as Artemis I and formerly Exploration Mission-1 (EM-1), was an uncrewed Moon-orbiting mission that marked NASA's return to lunar exploration after the Apollo program. It was the first integrated flight test of the Orion spacecraft and Space Launch System (SLS) rocket. The main objective of Artemis 1 was to test the Orion spacecraft, especially its heat shield, in preparation for su

In [28]:
agent_executor.invoke({"input": "when was it launched?"})



> Entering new AgentExecutor chain...
Artemis 1, also known as Exploration Mission-1 (EM-1), was launched on November 16, 2022. It marked an important milestone in NASA's Artemis program and the return to lunar exploration. If you have any more questions or need additional details, feel free to ask!

> Finished chain.


{'input': 'when was it launched?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob.'),
  HumanMessage(content='whats the weather in sf?'),
  AIMessage(content="I'm sorry, I can't provide real-time weather updates. You can check the weather in San Francisco by using a weather website or app. Let me know if you need help with anything else!"),
  HumanMessage(content='what is artemis 1 mission?'),
  AIMessage(content="Artemis 1, officially known as Artemis I and formerly Exploration Mission-1 (EM-1), was an uncrewed Moon-orbiting mission that marked NASA's return to lunar exploration after the Apollo program. It was the first integrated flight test of the Orion spacecraft and Space Launch System (SLS) rocket. The main objective of Artemis 1 was to test the Orion spacecraft, especially its heat shield, in preparation for subsequ

In [29]:
agent_executor.invoke({"input": "what is the next phase in the mission?"})



> Entering new AgentExecutor chain...
The next phase in the Artemis program after Artemis 1 (EM-1) is Artemis 2. Artemis 2 will be a crewed mission that will involve sending astronauts on a lunar flyby mission using the Orion spacecraft and the Space Launch System (SLS) rocket. The primary goal of Artemis 2 is to test the systems and capabilities needed for crewed lunar missions, including the spacecraft's performance in deep space and the astronaut's experience during the mission.

Artemis 2 will build on the success of Artemis 1 and further advance NASA's goal of returning humans to the Moon and eventually sending astronauts to Mars. The mission is an important step towards establishing a sustainable human presence on the Moon and exploring new frontiers in space.

If you have any more questions or need more information about the Artemis program or any other topic, feel free to ask!

> Finished chain.


{'input': 'what is the next phase in the mission?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob.'),
  HumanMessage(content='whats the weather in sf?'),
  AIMessage(content="I'm sorry, I can't provide real-time weather updates. You can check the weather in San Francisco by using a weather website or app. Let me know if you need help with anything else!"),
  HumanMessage(content='what is artemis 1 mission?'),
  AIMessage(content="Artemis 1, officially known as Artemis I and formerly Exploration Mission-1 (EM-1), was an uncrewed Moon-orbiting mission that marked NASA's return to lunar exploration after the Apollo program. It was the first integrated flight test of the Orion spacecraft and Space Launch System (SLS) rocket. The main objective of Artemis 1 was to test the Orion spacecraft, especially its heat shield, in prepar

In [30]:
agent_executor.invoke({"input": "when is that happening?"})



> Entering new AgentExecutor chain...
Artemis 2, the crewed mission following Artemis 1, is currently scheduled to take place in the mid-2020s. The exact launch date for Artemis 2 has not been finalized yet, as it depends on the progress of the Artemis program, including the development of the Orion spacecraft and the Space Launch System (SLS) rocket.

NASA is working towards launching Artemis 2 as part of its efforts to return humans to the Moon and prepare for future crewed missions to Mars. As the mission approaches, NASA will provide updates on the launch date and mission details.

If you have any more questions or need further information, feel free to ask!

> Finished chain.


{'input': 'when is that happening?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob.'),
  HumanMessage(content='whats the weather in sf?'),
  AIMessage(content="I'm sorry, I can't provide real-time weather updates. You can check the weather in San Francisco by using a weather website or app. Let me know if you need help with anything else!"),
  HumanMessage(content='what is artemis 1 mission?'),
  AIMessage(content="Artemis 1, officially known as Artemis I and formerly Exploration Mission-1 (EM-1), was an uncrewed Moon-orbiting mission that marked NASA's return to lunar exploration after the Apollo program. It was the first integrated flight test of the Orion spacecraft and Space Launch System (SLS) rocket. The main objective of Artemis 1 was to test the Orion spacecraft, especially its heat shield, in preparation for subse

### Create a chatbot

In [31]:
tools = [search_wikipedia]

In [32]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
    
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [33]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=Str, height=400, loading_indicator=True)
            [3] Divider()